In [6]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import scipy.stats as st
import matplotlib.pyplot as plt
import seaborn as sn
import os
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
from scipy.stats import ttest_ind
from statsmodels.stats.multitest import multipletests
print(os.getcwd())

g:\Github\Machine Learning\Machine-Learning-\ML in Autoimmune Diseases\Gene Method


In [7]:
ra_data = pd.read_csv("metadata_edited.csv")
ra_data_genes = pd.read_csv("arthritis.csv")
ra_data_genes.head()

,Patients,Condition,ARRDC5,FAM86C1,RAB3D,MORC2,ZNF600,SNAPC5,ZNF675,SFT2D1,...,LOC728534,SNORA24,CCT6P1,CCT6P3,SNORA22,RPL32P3,SNORA7A,SNORA7B,SNORA70,SNORA70G
0,GSM2981057,Healthy,5.598697,4.046514,4.792378,4.889926,6.614537,6.850238,3.627961,9.267813,...,4.782302,6.154111,5.863219,5.863219,5.863219,6.764443,6.764443,6.764443,5.183177,5.183177
1,GSM2981058,Healthy,5.566074,3.927057,4.885309,4.317204,6.634278,7.367398,4.292509,9.337131,...,4.944751,5.920788,6.022412,6.022412,6.022412,7.141088,7.141088,7.141088,4.940929,4.940929
2,GSM2981059,Healthy,5.102207,4.436170,4.440248,5.011531,7.082590,6.813552,4.041210,9.182283,...,4.188737,6.326932,5.573319,5.573319,5.573319,7.089928,7.089928,7.089928,4.777421,4.777421
3,GSM2981060,Healthy,4.855702,3.573448,4.805659,5.458887,7.385368,7.331518,3.712238,8.804739,...,3.931559,5.777944,6.063390,6.063390,6.063390,6.930789,6.930789,6.930789,4.906627,4.906627
4,GSM2981061,Healthy,5.151591,4.228044,4.906295,4.540327,7.299135,7.191299,3.749823,9.475676,...,4.856008,5.650359,6.103134,6.103134,6.103134,7.140819,7.140819,7.140819,4.924967,4.924967


In [8]:
healthy_data = ra_data_genes[ra_data_genes['Condition'] == 'Healthy']
unhealthy_data = ra_data_genes[ra_data_genes['Condition'] == 'Unhealthy']

In [9]:
# Extract the gene names and initialize a DataFrame to store p-values
genes = ra_data_genes.columns[2:]  # Assuming gene columns start from the third column

# Create a dictionary to store gene names and their corresponding p-values
gene_p_values = {'Gene': [], 'P-Value': []}

# Perform t-tests for each gene and store the results in the dictionary
for gene in genes:
    p_value = ttest_ind(healthy_data[gene], unhealthy_data[gene]).pvalue
    gene_p_values['Gene'].append(gene)
    gene_p_values['P-Value'].append(p_value)

# Create a DataFrame from the dictionary
p_values_data = pd.DataFrame(gene_p_values)

In [10]:
# Perform multiple testing correction (FDR correction)
corrected_p_values = multipletests(p_values_data['P-Value'], method='fdr_bh')[1]

# Add corrected p-values to the DataFrame
p_values_data['Corrected P-Value'] = corrected_p_values

In [11]:
significance_threshold = 0.0001
significant_genes = p_values_data[p_values_data['Corrected P-Value'] < significance_threshold]

In [12]:
# Save the significant genes to a new CSV file
significant_genes.to_csv('significant_genes.csv', index=False)

In [13]:
significant_genes.head()

,Gene,P-Value,Corrected P-Value
4,ZNF600,7.961667e-06,4.261581e-05
6,ZNF675,2.999705e-06,1.904525e-05
7,SFT2D1,5.352599e-09,1.349152e-07
9,DEPDC5,4.514808e-09,1.181645e-07
15,ZNF680,3.509774e-09,9.821967e-08


In [15]:
significant_genes.to_csv('significant_genes.csv', index=False)